<a href="https://colab.research.google.com/github/Tam02112003/seallm_7b_Q4_K_M_gguf/blob/main/seallm_7b_Q4_K_M_gguf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-cpp-python fastapi nest_asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067763 sha256=7fc6af9212382c8d4672b445f6e10286eecf40da148b8b2ade83b15ac0a5871d
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
!wget -O seallm-7b-v2.5-chatml.Q4_K_M.gguf \
"https://huggingface.co/SeaLLMs/SeaLLM-7B-v2.5-GGUF/resolve/main/seallm-7b-v2.5-chatml.Q4_K_M.gguf"


--2025-06-02 04:04:40--  https://huggingface.co/SeaLLMs/SeaLLM-7B-v2.5-GGUF/resolve/main/seallm-7b-v2.5-chatml.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 108.138.246.71, 108.138.246.85, 108.138.246.79, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.71|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/5c/bb/5cbb401e185f2f5a1716d0b4bedf7acd5b6dd6fc10ec6ed1451232464aa0465d/492de5493ec3310321f0d45dcd22a9734161e8d3e599740b59dcd678df1b33ca?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27seallm-7b-v2.5-chatml.Q4_K_M.gguf%3B+filename%3D%22seallm-7b-v2.5-chatml.Q4_K_M.gguf%22%3B&Expires=1748840680&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODg0MDY4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzVjL2JiLzVjYmI0MDFlMTg1ZjJmNWExNzE2ZDBiNGJlZGY3YWNkNWI2ZGQ2ZmMxMGVjNmVkMTQ1MTIzMjQ2NGFhMDQ2NWQvNDkyZGU1NDkzZWMzMzEwMzIxZjBkNDVkY2QyMmE5

In [ ]:
from fastapi import FastAPI, Request
from llama_cpp import Llama
import nest_asyncio
from pyngrok import ngrok, conf
import os

# Kích hoạt nest_asyncio để chạy FastAPI trong Colab
nest_asyncio.apply()

# Thiết lập token Ngrok
NGROK_AUTH_TOKEN = "2xjDlXKnK4ziZpXDJyVUtEOkPQn_d3jByZtxuNcPqeKoF2Tw"  # Thay bằng token ngrok thật
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Tạo FastAPI app
app = FastAPI()

# Tải mô hình SeaLLM (chatml format)
llm = Llama(
    model_path="seallm-7b-v2.5-chatml.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=8,   # tăng thread nếu CPU khỏe
    n_gpu_layers=40,  # giảm nếu GPU yếu, tăng nếu GPU mạnh
    f16_kv=True,
    use_mlock=True,
    use_cuda=True  # bật GPU nếu được
)


@app.post("/chat")
async def chat(request: Request):
    data = await request.json()
    context = data.get("context", "")
    prompt = data.get("prompt", "")

    # Định dạng ChatML
    messages = [
        {"role": "system", "content": "Bạn là trợ lý pháp lý thông minh, giúp người dùng hiểu rõ văn bản pháp lý."},
        {"role": "user", "content": f"""Đây là nội dung văn bản:\n{context}\n\nCâu hỏi: {prompt}"""}
    ]

    # Chuyển thành định dạng ChatML
    chatml_prompt = ""
    for m in messages:
        chatml_prompt += f"<|im_start|>{m['role']}{m['content']}<|im_end|>"
    chatml_prompt += "<|im_start|>assistant"

    res = llm.create_completion(
        prompt=chatml_prompt,
        max_tokens=128,
        temperature=0.7,
        top_p=0.9,
        stop=["<|im_end|>"]
    )

    output = res["choices"][0]["text"].strip()
    return {"response": output}


llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from seallm-7b-v2.5-chatml.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = step-149760_export_chatml
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 24576
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 16
llama_model_loader: - kv   7:              gemma.attention.he

In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

# Cho phép FastAPI hoạt động trong Colab
nest_asyncio.apply()

# Tạo tunnel ngrok
ngrok.kill()  # Đóng các tunnel cũ nếu có
public_url = ngrok.connect(8000)
print(f"🔗 Ngrok public URL: {public_url}")


uvicorn.run(app, host="0.0.0.0", port=8000)


🔗 Ngrok public URL: NgrokTunnel: "https://2643-34-125-205-22.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [446]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
llama_perf_context_print:        load time =  276530.01 ms
llama_perf_context_print: prompt eval time =  276527.35 ms /  1183 tokens (  233.75 ms per token,     4.28 tokens per second)
llama_perf_context_print:        eval time =   10789.15 ms /    14 runs   (  770.65 ms per token,     1.30 tokens per second)
llama_perf_context_print:       total time =  287358.09 ms /  1197 tokens


INFO:     125.235.122.164:0 - "POST /chat HTTP/1.1" 200 OK


Llama.generate: 1163 prefix-match hit, remaining 22 prompt tokens to eval
llama_perf_context_print:        load time =  276530.01 ms
llama_perf_context_print: prompt eval time =    5548.98 ms /    22 tokens (  252.23 ms per token,     3.96 tokens per second)
llama_perf_context_print:        eval time =   48775.23 ms /    64 runs   (  762.11 ms per token,     1.31 tokens per second)
llama_perf_context_print:       total time =   54490.87 ms /    86 tokens


INFO:     125.235.122.164:0 - "POST /chat HTTP/1.1" 200 OK


...

INFO:     125.235.122.164:0 - "POST /chat HTTP/1.1" 200 OK


Llama.generate: 1163 prefix-match hit, remaining 22 prompt tokens to eval
llama_perf_context_print:        load time =  276530.01 ms
llama_perf_context_print: prompt eval time =    6328.36 ms /    22 tokens (  287.65 ms per token,     3.48 tokens per second)
llama_perf_context_print:        eval time =   20320.09 ms /    27 runs   (  752.60 ms per token,     1.33 tokens per second)
llama_perf_context_print:       total time =   26718.67 ms /    49 tokens


INFO:     125.235.122.164:0 - "POST /chat HTTP/1.1" 200 OK


Llama.generate: 1173 prefix-match hit, remaining 23 prompt tokens to eval
llama_perf_context_print:        load time =  276530.01 ms
llama_perf_context_print: prompt eval time =    7150.62 ms /    23 tokens (  310.90 ms per token,     3.22 tokens per second)
llama_perf_context_print:        eval time =   46555.96 ms /    61 runs   (  763.21 ms per token,     1.31 tokens per second)
llama_perf_context_print:       total time =   53860.76 ms /    84 tokens


INFO:     125.235.122.164:0 - "POST /chat HTTP/1.1" 200 OK
